In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import string
from sklearn.metrics import mean_absolute_error
%matplotlib inline
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia= SentimentIntensityAnalyzer()

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

In [ ]:
# Data Read in
Calendar = pd.read_csv('calendar.csv')
listings = pd.read_csv('listings.csv')
reviews  = pd.read_csv('reviews.csv')


In [ ]:
Calendar.head()

In [ ]:
listings.head()

In [ ]:
reviews.head()

In [ ]:
Calendar.head()

In [ ]:
# Calandar Transform
Calendar['date'] = pd.to_datetime(Calendar['date'], format = '%Y-%m-%d') #date
#Remove Dollar sign
Calendar['price'] = Calendar['price'].replace('[\$,]', '', regex=True).astype(float)

Calendar.head()
 

In [ ]:
#Check Price column
Calendar['price'].unique()



In [ ]:
#check availability %
vals = Calendar.available.value_counts()
(vals/Calendar.shape[0]).plot(kind="bar");

In [ ]:
vals

In [ ]:
## See pricing over time for 3 random listings

#first sample 3 listings 

ids = Calendar.listing_id.unique()


In [ ]:
random.seed(40)
Ids = list(random.choices(ids, k = 1)) 
Ids

In [ ]:
#then Check their price remove when not available

df_sample = Calendar.loc[Calendar['listing_id'].isin(Ids)]
df_sample = df_sample.loc[df_sample['available'] == 't']
df_sample

In [ ]:
df_sample

In [ ]:
df_sample_2 = df_sample.drop('available', axis = 1)

In [ ]:
df_sample_2

In [ ]:
df_sample_2['listing_id'] = df_sample_2['listing_id'].astype(str)

In [ ]:
df_sample_2.plot(x = 'date', y = 'price' ); ## Prices don't flactuate a lot through out the year

In [ ]:
Calendar.describe()

In [ ]:
## lets see which month is busy (where available = F) but first we need to see if when available is false is it occupied
Calendar[Calendar]

In [ ]:
df_sample = Calendar.loc[Calendar['available'] == 't']

In [ ]:
sample = df_sample.groupby(['listing_id', df_sample['date'].dt.to_period("M")]).agg({'price':'mean'}).reset_index()

In [ ]:
# See which month has high price
sample_2 = sample.groupby(['date']).agg({'price':'mean'}).reset_index()
sample_2


In [ ]:
sample_2.plot(x = 'date', y='price', kind = 'bar');

In [ ]:
#plot Average prices across the months
sample_2.plot(x='date', y='price', kind='bar', color='skyblue', figsize=(10, 6))
plt.xlabel('Month')
plt.ylabel('Avg Price')
plt.title('Average Price against Months')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
df_sample

In [ ]:
## Check which listing has a high month price differential

sample_diff = sample.groupby('listing_id')['price'].agg(lambda x: x.max() - x.min()).reset_index()
sample_diff.rename(columns={'price': 'price_difference'}, inplace=True)
sample_diff


In [ ]:
sample_diff.sort_values(by = 'price_difference', ascending = False)

In [ ]:
avg_change = sample_diff['price_difference'].mean()
min_change = sample_diff['price_difference'].min()
max_change = sample_diff['price_difference'].max()
DF_Sum = pd.DataFrame({
    'Average': [avg_change],
    'Min': [min_change],
    'Max': [max_change]
})
DF_Sum

In [ ]:
hist = sample_diff.hist(column='price_difference', figsize=(12, 6))
plt.axvline(x=sample_diff['price_difference'].mean(), color='red', linestyle='--', label='Average')
plt.title('Distribution of of Price Differences')
plt.xlabel('Price Diffrence')
plt.ylabel('Frequency')


plt.show()

In [ ]:
#Calendar.loc[Calendar['available'] == 't']
listings.loc[listings['id'] == 3308979]

In [ ]:
## See the reviews vs price
listings.head(5)

In [ ]:
#check which listings don't have reviews per month
listings['reviews_per_month'].isna().sum()/listings.shape[0]

In [ ]:
Calendar.groupby('listing_id').agg({'price':'mean'}).reset_index()

In [ ]:
#lets join listings with price because this port we plan on doing averege reviews per price
#first lets get average price for listings
calendar_avg = Calendar.groupby('listing_id').agg({'price':'mean'}).reset_index()
calendar_avg.rename(columns={'listing_id': 'id', 'price':'avg_price'}, inplace = True)
calendar_avg

In [ ]:
#join
listings_calendar = pd.merge(listings, calendar_avg, on = 'id', how = 'left')
listings_calendar

In [ ]:
listings_calendar.columns

In [ ]:
# Check review scores vs price
scores = listings_calendar[['id', 'name', 'description','review_scores_rating', 'reviews_per_month','avg_price']]
scores.head()

In [ ]:
scores['id'].nunique()/scores.shape[0] #no duplicates

In [ ]:
# Remove listings without price and rating
#first check percentage we would be losing
(scores[['review_scores_rating', 'avg_price']].isna().any(axis=1).sum() * 100)/scores.shape[0]

In [ ]:
#then remove rows
scores_clean = scores.dropna(subset=['review_scores_rating', 'avg_price'])


In [ ]:
#check
(scores_clean[['review_scores_rating', 'avg_price']].isna().any(axis=1).sum() * 100)/scores_clean.shape[0]

In [ ]:
# check scatter of price against reviews
plt.scatter(x= scores_clean['review_scores_rating'], y = scores_clean['avg_price'])
plt.xlabel('Review Scores')
plt.ylabel('Average Price')
plt.title('Review Scores against average price')

plt.show()


In [ ]:
#check correlation 
correlation = scores_clean['review_scores_rating'].corr(scores_clean['avg_price'])
correlation


In [ ]:
scores_clean['avg_price'].mean()

In [ ]:
buckets = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
scores_clean['Review_bucket'] = pd.cut(scores_clean['review_scores_rating'], bins=buckets)
scores_clean.head()

In [ ]:
#groupby('listing_id').agg({'price':'mean'}).reset_index()
review_per_price = scores_clean.groupby('Review_bucket')['avg_price'].mean().reset_index()

In [ ]:
review_per_price

In [ ]:
review_per_price.plot(x = 'Review_bucket', y = 'avg_price',  kind = 'bar');
plt.xlabel('Review Bucket')
plt.ylabel('Average Price')
plt.title('Average Price vs. Review Bucket') 

In [ ]:
#So the highest price average got and average rating score, not not all highly priced bnbs have high review scores. also nice to see the counts
counts = pd.DataFrame(scores_clean['Review_bucket'].astype(str).value_counts().reset_index())
counts.columns = ['Review_bucket', 'Counts']
counts


In [ ]:
scores_clean.dtypes

In [ ]:
#lets create box plot
#the_90s = scores_clean[scores_clean['Review_bucket'] == pd.Interval(90, 100, closed='right')]
plt.figure(figsize=(100, 6))
scores_clean.boxplot(column = 'avg_price', by = 'Review_bucket', showfliers=False, vert=True);
plt.title(f'Boxplot of avg_price against Buckets')
plt.xlabel('Buckets')
plt.ylabel('avg_price')
plt.suptitle('')
plt.show()

In [ ]:
#now lets try to check the sentiment of reviews to check against the price
reviews.head()

In [ ]:
#steps
# 1. remove stop words
# 2. Tokenise the comments (try unigrams, then bigrams (for negating words))
# 3. get sentiments
# 4. adjust for negation
listings_calendar.head()


In [ ]:
#select nessesary columns
reviews_1 = reviews[['listing_id','id','comments']]
reviews_1.head()


In [ ]:
#create function that tokenizes and removes stopwords/punctuation
def Tokenize (text, n = 2):
    text_no_pun = text.translate(str.maketrans("","", string.punctuation))#remove punctuation
    stop_words = set(stopwords.words('english')) #get stop word
    tokens = word_tokenize(text_no_pun)
    removed_stop = [word for word in tokens if word.lower() not in stop_words] #remove stop words
    bingrams = list(ngrams(removed_stop, n)) # bigrams
    return bingrams

In [ ]:
string.punctuation

In [ ]:
#Tokenize
reviews_1['comments'] = reviews_1['comments'].astype(str)
reviews_1['comments_bigrams'] = reviews_1['comments'].apply(Tokenize)
reviews_1

In [ ]:
reviews_final = reviews_1.explode('comments_bigrams').reset_index()
reviews_final

In [ ]:
reviews_final.drop('index', axis=1, inplace=True)

In [ ]:
reviews_final

In [ ]:
#Split bigrams 
reviews_final[['word_1', 'word_2']] = pd.DataFrame(reviews_final['comments_bigrams'].tolist(), index=reviews_final.index)

In [ ]:
reviews_final

In [ ]:
#Check which words are common
reviews_final['word_1'].str.lower().mode()[0]

In [ ]:
common_words = pd.DataFrame(reviews_final['word_1'].value_counts().head(10))
common_words = common_words.rename(columns={'word_1': 'count', ' ': 'words'})
common_words


In [ ]:
#since doing sentiment is proving defficult due to foreign words lets use the scores and see which variables prove imported in avrege scoring
listings_calendar.columns
listings_calendar.shape[1]

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
#remove unessesary columns (names and descriptions)
Raw_Data = listings_calendar[['id', 'experiences_offered','host_location', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet',
       'price', 'weekly_price', 'monthly_price', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90', 'calendar_last_scraped', 'number_of_reviews',
       'first_review', 'last_review', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'requires_license',
       'license', 'jurisdiction_names', 'instant_bookable',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification', 'calculated_host_listings_count',
       'reviews_per_month', 'avg_price']]

In [ ]:
Raw_Data.shape[1]

In [ ]:
# lets check nulls to see what we remove 
NAS_percentage = (Raw_Data.isnull().mean() * 100).round(2)
NAS_percentage[NAS_percentage > 25] #license is completly null, other have high nulls, we will remove these columns with 25% high nulls especially with just 3.8 rows that would create a biased model

In [ ]:
NAS_percentage

In [ ]:
#drop columns NA greater than 25%
threshold = len(Raw_Data) * 0.25
Raw_Data_1 = Raw_Data.dropna(axis=1, thresh=threshold)

In [ ]:
raw_catagorical = Raw_Data_1.select_dtypes(include=['object', 'category'])

In [ ]:
raw_catagorical.head()

raw_catagorical.neighbourhood.unique()

#zip code is like the areas so we will see if we can get it else where

In [ ]:
#Checking for columns that have low variance or low unique values (for catagorical), we not removing binary columns
pd.set_option('display.max_columns', None)
Raw_Data_1.select_dtypes(include='number')

In [ ]:
dropped_columns = ['experiences_offered','host_acceptance_rate', 'city', 'state', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 'is_location_exact', 'has_availability', 
                   'calendar_last_scraped', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'calendar_updated', 'calendar_last_scraped', 'jurisdiction_names', 'first_review', 'last_review', 'host_listings_count',
                   'latitude', 'longitude', 'review_scores_accuracy', 'review_scores_cleanliness',  'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'calculated_host_listings_count', 'reviews_per_month',
                   'neighbourhood']

In [ ]:
len(dropped_columns)

In [ ]:
#lets see which neighbourhoods get the most average views. Can see we can remove neighbourhoods as it doesn't give any information
df_cleaned = Raw_Data_1.dropna(subset=['review_scores_rating', 'neighbourhood'])
df_sorted = df_cleaned.sort_values(by='review_scores_rating', ascending=False)
plt.figure(figsize=(100, 6))
plt.bar(df_sorted['neighbourhood'], df_sorted['review_scores_rating'], color='skyblue')
plt.xlabel('Neighborhood')
plt.ylabel('Review Score')
plt.title('Neighborhoods with the Highest Reviews')
plt.show()


In [ ]:
average_ratings = Raw_Data_1.groupby('neighbourhood')['review_scores_rating'].mean().reset_index()
average_ratings

In [ ]:
#which ameneties result in high reviews
Raw_Data_1.head()

columns = ['property_type','accommodates', 'bathrooms','bedrooms','beds','bed_type','amenities','avg_price','review_scores_rating']

In [ ]:
raw_ameneties = Raw_Data_1[columns]
raw_ameneties.head()

In [ ]:
#remove where Y is nan
raw_ameneties_1 = raw_ameneties.dropna(subset=['review_scores_rating'])
raw_ameneties_1.head()
raw_ameneties_1.shape

In [ ]:
#hot encode property and bed_type
raw_ameneties_1['bed_type'].unique()

In [ ]:
encoded_ameneties = pd.get_dummies(raw_ameneties_1, columns=['property_type', 'bed_type'], prefix=['property_type', 'bed_type'])

In [ ]:
encoded_ameneties.shape


In [ ]:
#extract amenities and encode
amenities_dummies = encoded_ameneties['amenities'].str.extractall(r'"(.*?)"')[0].str.get_dummies().max(level=0)
amenities_dummies
#append amenities
amenities_dummies_final = pd.concat([encoded_ameneties, amenities_dummies], axis=1)
amenities_dummies_final

In [ ]:
amenities_dummies_final = amenities_dummies_final.drop(columns = 'amenities')

In [ ]:
#reduced corr plot
amenities_dummies_final_1 = amenities_dummies_final[['bathrooms', 'bedrooms', 'beds', 'review_scores_rating', 'Buzzer/Wireless Intercom', 'Family/Kid Friendly', 'avg_price']]
 # Select only numeric columns for correlation matrix
numeric_columns = amenities_dummies_final_1.select_dtypes(include='number')

# Calculate the correlation matrix
correlation_matrix = numeric_columns.corr()

# Plot the heatmap
plt.figure(figsize=(10,5))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Correlation Matrix')
plt.show()



In [ ]:
 # Select only numeric columns for correlation matrix
numeric_columns = amenities_dummies_final.select_dtypes(include='number')

# Calculate the correlation matrix
correlation_matrix = numeric_columns.corr()

# Plot the heatmap
plt.figure(figsize=(50,20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Correlation Matrix')
plt.show()

#high correlation with accomodates and number of bedrooms, beds and bathrooms, 

In [ ]:
#drop accomodates and beds
amenities_dummies_final = amenities_dummies_final.drop(columns=['accommodates', 'beds'])
numeric_columns = amenities_dummies_final.select_dtypes(include='number')

# Calculate the correlation matrix
correlation_matrix = numeric_columns.corr()

# Plot the heatmap
plt.figure(figsize=(50,20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
#see other correlated variables and drop
upper = correlation_matrix.abs().where(np.triu(np.ones(correlation_matrix.abs().shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.70)]

In [ ]:
amenities_dummies_final.drop(to_drop, axis=1, inplace=True)
numeric_columns = amenities_dummies_final.select_dtypes(include='number')
# Calculate the correlation matrix
correlation_matrix = numeric_columns.corr()

# Plot the heatmap
plt.figure(figsize=(50,20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Correlation Matrix')
plt.show()